**Import necessary libraries and set up**

In [ ]:
!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

--2024-07-22 16:01:11--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 3.86.22.73
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|3.86.22.73|:80... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2024-07-22 16:01:11--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1191 (1.2K) [text/plain]
Saving to: ‘STDOUT’

-                   100%[===================>]   1.16K  --.-KB/s    in 0s      

2024-07-22 16:01:11 (54.6 MB/s) - written to stdout [1191/1191]

Installing PySpark

In [ ]:
import sparknlp
spark = sparknlp.start()

from sparknlp.pretrained import PretrainedPipeline

In [ ]:
pipeline = PretrainedPipeline("explain_document_ml")

explain_document_ml download started this may take some time.
Approx size to download 9 MB
[OK!]


In [1]:
import random
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/My Drive/Colab_Notebooks"

Mounted at /content/drive
CC_gender_Schwartz.ipynb  CC_HW8_Schwartz	cuckoos_calling_text.txt  hp2_text.txt
CC_HW1_Schwartz.ipynb	  combined_data.csv	final_ds.csv		  hp7_text.txt
CC_HW2_Schwartz.ipynb	  combined_data.gsheet	hp1_text.txt


**Read in text files**

In [ ]:
cc = open('/content/drive/My Drive/Colab_Notebooks/cuckoos_calling_text.txt').readlines()
hp1 = open('/content/drive/My Drive/Colab_Notebooks/hp1_text.txt').readlines()
hp7 = open('/content/drive/My Drive/Colab_Notebooks/hp7_text.txt').readlines()


**Select random specified number of lines from each file**

In [ ]:
random.seed()
cc_red = random.sample(cc,7500)
hp1_red = random.sample(hp1, 9149)
h7_red = random.sample(hp7, 8550)


**Find parts of speech**

*Cuckoo's Calling*

In [ ]:
#these lines are mostly from lecture, only with some of my own additions

#convert to dataframe for sparkpy
df = pd.DataFrame(cc_red, columns=['text'])
df.iteritems = df.items #error with below line unless renamed it as such. had to do with package versions
data = spark.createDataFrame(df)
dfs = pipeline.transform(data)

In [ ]:
#these lines are from lecture, not written by myself

# Extract words and parts-of-speech
dfs_cc = [pipeline.annotate(hl) for hl in cc_red]
tok_tag_cc = [(df['token'],df['pos']) for df in dfs_cc]

# fuse pos to word
zips_cc = [list(zip(tt[0], tt[1])) for tt in tok_tag_cc]
print(zips_cc)


[[('Charlie', 'NNP'), (',', ','), ('and', 'CC'), ('I', 'PRP'), ('wanted', 'VBD'), ('a', 'DT'), ('daughter', 'NN'), ('so', 'RB'), ('badly', 'RB'), (';', ':'), ('the', 'DT'), ('idea', 'NN'), ('of', 'IN'), ('losing', 'VBG'), ('her', 'PRP$'), (',', ','), ('too…”', 'NN')], [('Roehampton', 'NNP'), ('.', '.'), ('Matthew', 'NNP'), ('was', 'VBD'), ('taking', 'VBG'), ('her', 'PRP'), ('to', 'TO'), ('Yorkshire', 'NNP'), ('for', 'IN'), ('the', 'DT'), ('weekend', 'NN'), ('.', '.')], [('This', 'DT'), ('drew', 'VBD'), ('a', 'DT'), ('small', 'JJ'), ('snort', 'NN'), ('from', 'IN'), ('Rochelle', 'NNP'), ('.', '.')], [('“Did', 'VBD'), ('you', 'PRP'), ('meet', 'VBP'), ('Evan', 'NNP'), ('Duffield?”', 'NNP'), ('Strike', 'NNP'), ('asked', 'VBD'), ('.', '.')], [('tell', 'VB'), ('now', 'RB'), (',', ','), ('a', 'DT'), ('funny', 'JJ'), ('family', 'NN'), ('story', 'NN'), (',', ','), ('the', 'DT'), ('kind', 'NN'), ('you', 'PRP'), ('told', 'VBD'), ('your', 'PRP$'), ('children', 'NNS'), (',', ','), ('in', 'IN'), ('wh

In [ ]:
# Filter for specific pronouns
pronouns = ["she", "her", "hers", "he", "his", "him", "She", "Her", "Hers", "He", "His", "Him"]

In [ ]:
# Filter for specific pronouns
filtered_zips_cc = [[(word, pos) for word, pos in zip_pair if word.lower() in pronouns] for zip_pair in zips_cc]


In [ ]:
#find frequency of each pronoun's part of speech

# Flatten the list of lists
flat_filtered_zips_cc = [item for sublist in filtered_zips_cc for item in sublist]

# Create a list to store the data for the df
data_cc = []
for word, pos in flat_filtered_zips_cc:
  data_cc.append({'Pronoun': word.lower(), 'Part of Speech': pos})

# Create df
df_cc = pd.DataFrame(data_cc)

# Create pivot table
pivot_table_cc = df_cc.pivot_table(index='Pronoun', columns='Part of Speech', aggfunc='size', fill_value=0)
print(pivot_table_cc)


Part of Speech  NNS   PRP  PRP$
Pronoun                        
he                0  1150     0
her               0   137   758
hers              2     0     0
him               0   314     0
his               0     0   745
she               0   956     0


*Harry Potter 1*

In [ ]:
#repeat steps from above

#convert to dataframe for sparkpy
df_hp1 = pd.DataFrame(hp1_red, columns=['text'])
df_hp1.iteritems = df_hp1.items #error with below line unless renamed it as such. had to do with package versions
data_hp1 = spark.createDataFrame(df_hp1)
dfs_hp1 = pipeline.transform(data_hp1)

In [ ]:
# Extract words and parts-of-speech
dfs_hp1 = [pipeline.annotate(hl) for hl in hp1_red]

In [ ]:
# Extract words and parts-of-speech
tok_tag_hp1 = [(df_hp1['token'],df_hp1['pos']) for df_hp1 in dfs_hp1]

# fuse pos to word
zips_hp1 = [list(zip(tt[0], tt[1])) for tt in tok_tag_hp1]
print(zips_hp1)

[[], [('squeaked', 'VBD'), ('Professor', 'NNP'), ('Flitwick', 'NNP'), (',', ','), ('perched', 'VBN'), ('on', 'IN'), ('top', 'NN'), ('of', 'IN'), ('his', 'PRP$'), ('pile', 'NN'), ('of', 'IN'), ('books', 'NNS'), ('as', 'IN')], [], [], [('looked', 'VBD'), ('over', 'IN'), ('her', 'PRP$'), ('shoulder', 'NN'), ('to', 'TO'), ('read', 'VB'), ('it', 'PRP'), (':', ':')], [('"', "''"), ("How's", 'NNP'), ('your', 'PRP$'), ('arm', 'NN'), ('?"', 'NN'), ('said', 'VBD'), ('Harry', 'NNP'), ('.', '.')], [('26', 'CD')], [('218', 'CD')], [('"', "''"), ('Come', 'VBN'), ('on', 'IN'), (',', ','), ('Hermione', 'NNP'), (',"', 'NN'), ('Ron', 'NNP'), ('muttered', 'JJ'), ('desperately', 'RB'), ('.', '.')], [('slope', 'NN'), (',', ','), ('"', "''"), ('that', 'IN'), ('what', 'WP'), ('Hagrid', 'NN'), ('wants', 'VBZ'), ('more', 'JJR'), ('than', 'IN'), ('anything', 'NN'), ('else', 'RB'), ('is', 'VBZ'), ('a', 'DT'), ('dragon', 'NN'), (',', ','), ('and', 'CC')], [('"', "''"), ('No', 'DT'), ('--', ':'), ("I'm", 'NNP'), (

In [ ]:
# Filter for specific pronouns
filtered_zips_hp1 = [[(word, pos) for word, pos in zip_pair if word.lower() in pronouns] for zip_pair in zips_hp1]

In [ ]:
#find frequency of each pronoun's part of speech

# Flatten the list of lists
flat_filtered_zips_hp1 = [item for sublist in filtered_zips_hp1 for item in sublist]

# Create a list to store the data for the df
data_hp1b = []
for word, pos in flat_filtered_zips_hp1:
  data_hp1b.append({'Pronoun': word.lower(), 'Part of Speech': pos})

# Create df
df_hp1b = pd.DataFrame(data_hp1b)

# Create pivot table
pivot_table_hp1 = df_hp1b.pivot_table(index='Pronoun', columns='Part of Speech', aggfunc='size', fill_value=0)
print(pivot_table_hp1)

Part of Speech   PRP  PRP$
Pronoun                   
he              1268     0
her               24   126
him              404     0
his                0   792
she              194     0


*Harry Potter 7*

In [ ]:
#repeat steps from above

#convert to dataframe for sparkpy
df_hp7 = pd.DataFrame(h7_red, columns=['text'])
df_hp7.iteritems = df_hp7.items #error with below line unless renamed it as such. had to do with package versions
data_hp7 = spark.createDataFrame(df_hp7)
dfs_hp7 = pipeline.transform(data_hp7)

In [ ]:
# Extract words and parts-of-speech
dfs_hp7 = [pipeline.annotate(hl) for hl in h7_red]

In [ ]:
# Extract words and parts-of-speech
tok_tag_hp7 = [(df_hp7['token'],df_hp7['pos']) for df_hp7 in dfs_hp7]

# fuse pos to word
zips_hp7 = [list(zip(tt[0], tt[1])) for tt in tok_tag_hp7]
print(zips_hp7)

[[('okay', 'JJ'), (',', ','), ('Ron—”', 'NNP')], [('“Thought', 'NNP'), ('you', 'PRP'), ('didn’t', 'NN'), ('approve', 'NN'), ('of', 'IN'), ('that', 'DT'), ('spell?”', 'NN'), ('said', 'VBD'), ('Ron', 'NNP'), ('.', '.')], [('As', 'IN'), ('he', 'PRP'), ('crossed', 'VBD'), ('the', 'DT'), ('dark', 'JJ'), ('yard', 'NN'), (',', ','), ('the', 'DT'), ('great', 'JJ'), ('skeletal', 'NN'), ('thestral', 'NN'), ('looked', 'VBD')], [('protection', 'NN'), (',', ','), ('the', 'DT'), ('best', 'JJS'), ('there', 'EX'), ('is', 'VBZ'), ('.', '.'), ('”', 'NN')], [('Slightly', 'RB'), ('taken', 'VBN'), ('aback', 'NN'), (',', ','), ('Harry', 'NNP'), ('hesitated', 'JJ'), ('.', '.')], [('“King’s', 'NN'), ('Cross', 'NNP'), ('station!”', 'NN'), ('Dumbledore', 'NN'), ('was', 'VBD'), ('chuckling', 'VBG'), ('immoder', 'NN'), ('-', '-')], [('questioning', 'VBG'), ('today', 'NN'), ('?', '.'), ('Er—what’s', 'NNP'), ('happened', 'VBD'), ('to', 'TO'), ('you', 'PRP'), ('?', '.'), ('Why', 'WRB'), ('are', 'VBP'), ('you', 'PRP'

In [ ]:
# Filter for specific pronouns
filtered_zips_hp7 = [[(word, pos) for word, pos in zip_pair if word.lower() in pronouns] for zip_pair in zips_hp7]

In [ ]:
#find frequency of each pronoun's part of speech

# Flatten the list of lists
flat_filtered_zips_hp7 = [item for sublist in filtered_zips_hp7 for item in sublist]

# Create a list to store the data for the df
data_hp7b = []
for word, pos in flat_filtered_zips_hp7:
  data_hp7b.append({'Pronoun': word.lower(), 'Part of Speech': pos})

# Create df
df_hp7b = pd.DataFrame(data_hp7b)

# Create pivot table
pivot_table_hp7 = df_hp7b.pivot_table(index='Pronoun', columns='Part of Speech', aggfunc='size', fill_value=0)
print(pivot_table_hp7)

Part of Speech  NNP  NNS   PRP  PRP$
Pronoun                             
he                0    0  1399     0
her               2    0    47   323
hers              0    1     0     0
him               0    0   441     0
his               0    0     0   907
she               0    0   290     0


**Find proportions, normalize**



*Cuckoo's Calling*

In [ ]:
# CC

# Calculate the 'total' for each row (already done, but including for completeness)
pivot_table_cc["total"] = pivot_table_cc.sum(axis=1)

# Normalize each row by dividing by its total
pivot_table_cc_normalized = pivot_table_cc.div(pivot_table_cc["total"], axis=0)

# Drop the 'total' column as it's no longer needed
pivot_table_cc_normalized = pivot_table_cc_normalized.drop('total', axis=1)

# Transpose the normalized pivot table
pivot_table_cc_transposed = pivot_table_cc_normalized.transpose()

# Generate column names as expected (24 columns)
expected_columns = [f"{pos}_{pronoun}" for pronoun in pivot_table_cc_transposed.index
                   for pos in pivot_table_cc_transposed.columns]

# Create a new DataFrame with a single row for 'CC'
# Reshape the values to a single row with 24 columns
cc_row = pd.DataFrame(pivot_table_cc_transposed.values.reshape(1,-1),
                     index=['CC'],
                     columns=expected_columns) # Use the generated column names

print(cc_row)


    he_NNS  her_NNS  hers_NNS  him_NNS  his_NNS  she_NNS  he_PRP   her_PRP  \
CC     0.0      0.0       1.0      0.0      0.0      0.0     1.0  0.153073   

    hers_PRP  him_PRP  his_PRP  she_PRP  he_PRP$  her_PRP$  hers_PRP$  \
CC       0.0      1.0      0.0      1.0      0.0  0.846927        0.0   

    him_PRP$  his_PRP$  she_PRP$  
CC       0.0       1.0       0.0  


*Harry Potter 1*

In [ ]:
# HP1

# Calculate the 'total' for each row (already done, but including for completeness)
pivot_table_hp1["total"] = pivot_table_hp1.sum(axis=1)

# Normalize each row by dividing by its total
pivot_table_hp1_normalized = pivot_table_hp1.div(pivot_table_hp1["total"], axis=0)

# Drop the 'total' column as it's no longer needed
pivot_table_hp1_normalized = pivot_table_hp1_normalized.drop('total', axis=1)

# Transpose the normalized pivot table
pivot_table_hp1_transposed = pivot_table_hp1_normalized.transpose()

# Generate column names as expected (24 columns)
expected_columns = [f"{pos}_{pronoun}" for pronoun in pivot_table_hp1_transposed.index
                   for pos in pivot_table_hp1_transposed.columns]

# Create a new DataFrame with a single row for 'hp1'
# Reshape the values to a single row with 24 columns
hp1_row = pd.DataFrame(pivot_table_hp1_transposed.values.reshape(1,-1),
                     index=['HP1'],
                     columns=expected_columns) # Use the generated column names

print(hp1_row)

     he_PRP  her_PRP  him_PRP  his_PRP  she_PRP  he_PRP$  her_PRP$  him_PRP$  \
HP1     1.0     0.16      1.0      0.0      1.0      0.0      0.84       0.0   

     his_PRP$  she_PRP$  
HP1       1.0       0.0  


*Harry Potter 7*

In [ ]:
# HP7

# Calculate the 'total' for each row (already done, but including for completeness)
pivot_table_hp7["total"] = pivot_table_hp7.sum(axis=1)

# Normalize each row by dividing by its total
pivot_table_hp7_normalized = pivot_table_hp7.div(pivot_table_hp7["total"], axis=0)

# Drop the 'total' column as it's no longer needed
pivot_table_hp7_normalized = pivot_table_hp7_normalized.drop('total', axis=1)

# Transpose the normalized pivot table
pivot_table_hp7_transposed = pivot_table_hp7_normalized.transpose()

# Generate column names as expected (24 columns)
expected_columns = [f"{pos}_{pronoun}" for pronoun in pivot_table_hp7_transposed.index
                   for pos in pivot_table_hp7_transposed.columns]

# Create a new DataFrame with a single row for 'hp7'
# Reshape the values to a single row with 24 columns
hp7_row = pd.DataFrame(pivot_table_hp7_transposed.values.reshape(1,-1),
                     index=['HP7'],
                     columns=expected_columns) # Use the generated column names

print(hp7_row)

     he_NNP   her_NNP  hers_NNP  him_NNP  his_NNP  she_NNP  he_NNS  her_NNS  \
HP7     0.0  0.005376       0.0      0.0      0.0      0.0     0.0      0.0   

     hers_NNS  him_NNS  ...  hers_PRP  him_PRP  his_PRP  she_PRP  he_PRP$  \
HP7       1.0      0.0  ...       0.0      1.0      0.0      1.0      0.0   

     her_PRP$  hers_PRP$  him_PRP$  his_PRP$  she_PRP$  
HP7   0.86828        0.0       0.0       1.0       0.0  

[1 rows x 24 columns]


*Combine*

In [ ]:
# Concatenate the rows into a single DataFrame
combined_df = pd.concat([cc_row, hp1_row, hp7_row])

# Fill missing columns with 0.0
combined_df = combined_df.fillna(0.0)

print(combined_df)

     he_NNS  her_NNS  hers_NNS  him_NNS  his_NNS  she_NNS  he_PRP   her_PRP  \
CC      0.0      0.0       1.0      0.0      0.0      0.0     1.0  0.153073   
HP1     0.0      0.0       0.0      0.0      0.0      0.0     1.0  0.160000   
HP7     0.0      0.0       1.0      0.0      0.0      0.0     1.0  0.126344   

     hers_PRP  him_PRP  ...  hers_PRP$  him_PRP$  his_PRP$  she_PRP$  he_NNP  \
CC        0.0      1.0  ...        0.0       0.0       1.0       0.0     0.0   
HP1       0.0      1.0  ...        0.0       0.0       1.0       0.0     0.0   
HP7       0.0      1.0  ...        0.0       0.0       1.0       0.0     0.0   

      her_NNP  hers_NNP  him_NNP  his_NNP  she_NNP  
CC   0.000000       0.0      0.0      0.0      0.0  
HP1  0.000000       0.0      0.0      0.0      0.0  
HP7  0.005376       0.0      0.0      0.0      0.0  

[3 rows x 24 columns]


*Clean*



In [ ]:
#save just in case
combined_df.to_csv('/content/drive/My Drive/Colab_Notebooks/combined_data.csv', index=True)


In [34]:
#read in if starting here
combined_df = pd.read_csv('/content/drive/My Drive/Colab_Notebooks/combined_data.csv')


In [35]:
#some cleaning

combined_df = combined_df.rename(columns={combined_df.columns[0]: 'books'}) #first col wasn't labeled

combined_df = combined_df.filter(regex='^(her_|him_|his_|hers_|books)') #keep only necessary columns

combined_df = combined_df.set_index('books').T #transpose so can divide easier

combined_df

books,CC,HP1,HP7
her_NNS,0.000000,0.00,0.000000
hers_NNS,1.000000,0.00,1.000000
him_NNS,0.000000,0.00,0.000000
his_NNS,0.000000,0.00,0.000000
her_PRP,0.153073,0.16,0.126344
hers_PRP,0.000000,0.00,0.000000
him_PRP,1.000000,1.00,1.000000
his_PRP,0.000000,0.00,0.000000
her_PRP$,0.846927,0.84,0.868280
hers_PRP$,0.000000,0.00,0.000000


In [36]:
#normalize to HP1
#don't forget 0s were replaced with 0.00001

for col in combined_df.columns:
      combined_df[col + '_norm'] = np.where(
          (combined_df['HP1'] == 0) & (combined_df[col] == 0), 1,
          np.where(
              (combined_df['HP1'] == 0) & (combined_df[col] != 0), combined_df[col],
              combined_df[col] / combined_df['HP1']
          )
      )

combined_df


books,CC,HP1,HP7,CC_norm,HP1_norm,HP7_norm
her_NNS,0.000000,0.00,0.000000,1.000000,1.0,1.000000
hers_NNS,1.000000,0.00,1.000000,1.000000,1.0,1.000000
him_NNS,0.000000,0.00,0.000000,1.000000,1.0,1.000000
his_NNS,0.000000,0.00,0.000000,1.000000,1.0,1.000000
her_PRP,0.153073,0.16,0.126344,0.956704,1.0,0.789651
hers_PRP,0.000000,0.00,0.000000,1.000000,1.0,1.000000
him_PRP,1.000000,1.00,1.000000,1.000000,1.0,1.000000
his_PRP,0.000000,0.00,0.000000,1.000000,1.0,1.000000
her_PRP$,0.846927,0.84,0.868280,1.008247,1.0,1.033666
hers_PRP$,0.000000,0.00,0.000000,1.000000,1.0,1.000000


In [37]:
#finish cleaning for ease of comparison

combined_df=combined_df.drop(columns=['CC', 'HP1', 'HP7']) #drop unnecessary columns
combined_df = combined_df.T #put back, easier to visualize
combined_df = combined_df.loc[:, ~combined_df.eq(1).all(axis=0)] #select only ones that are different

In [38]:
combined_df

,her_PRP,her_PRP$,her_NNP
books,,,
CC_norm,0.956704,1.008247,1.000000
HP1_norm,1.000000,1.000000,1.000000
HP7_norm,0.789651,1.033666,0.005376


**Save for data visualization in R**

In [39]:
combined_df.to_csv('/content/drive/My Drive/Colab_Notebooks/final_ds.csv', index=True)